## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [13]:
import os
from keras.datasets import cifar10
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [14]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [15]:
# 建立 ResNet 模型
model = resnet(input_shape=(32,32,3)) 
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_63 (Conv2D)              (None, 32, 32, 16)   448         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_57 (BatchNo (None, 32, 32, 16)   64          conv2d_63[0][0]                  
__________________________________________________________________________________________________
activation_57 (Activation)      (None, 32, 32, 16)   0           batch_normalization_57[0][0]     
__________________________________________________________________________________________________
conv2d_64 

In [16]:
batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練整個資料集共 30個循環
LEARNING_RATE = 1e-3

## 加入augmentation

In [17]:
train_datagen = ImageDataGenerator(rotation_range=20, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   horizontal_flip=True )

In [18]:
"""
# 載入 Callbacks, 並設定監控目標為 validation loss
"""

reduce_lr = ReduceLROnPlateau(factor=0.5, 
                              min_lr=1e-12, 
                              monitor='val_loss', 
                              patience=5, 
                              verbose=1)

In [21]:
optimizer_ = Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [22]:

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer_,
              metrics=['accuracy'])

history = model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size, shuffle=True),
                              steps_per_epoch=len(x_train)//batch_size,
                              epochs=epochs,
                              verbose=1,
                              validation_data=(x_test, y_test),
                             callbacks=[reduce_lr])
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/30
781/781 [==============================] - 35s 45ms/step - loss: 2.0162 - acc: 0.4375 - val_loss: 1.9689 - val_acc: 0.4438
Epoch 2/30
781/781 [==============================] - 29s 37ms/step - loss: 1.5748 - acc: 0.5584 - val_loss: 1.9432 - val_acc: 0.4631
Epoch 3/30
781/781 [==============================] - 29s 37ms/step - loss: 1.3724 - acc: 0.6171 - val_loss: 1.4602 - val_acc: 0.6028
Epoch 4/30
781/781 [==============================] - 29s 37ms/step - loss: 1.2486 - acc: 0.6595 - val_loss: 1.5935 - val_acc: 0.5852
Epoch 5/30
781/781 [==============================] - 29s 37ms/step - loss: 1.1467 - acc: 0.6893 - val_loss: 1.6145 - val_acc: 0.6076
Epoch 6/30
781/781 [==============================] - 29s 37ms/step - loss: 1.0783 - acc: 0.7148 - val_loss: 1.3317 - val_acc: 0.6709
Epoch 7/30
781/781 [==============================] - 29s 37ms/step - loss: 1.0207 - acc: 0.7337 - val_loss: 1.1301 - val_acc: 0.7097
Epoch 8/30
781/781 [==============================] - 29s 37ms

### 範例結果
* Test loss: 1.4235524982452392
* Test accuracy: 0.7548

### 實驗1 - 加入data augmentation

{rotation_range=20, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   horizontal_flip=True
}
* Test loss: 1.1394029851913452
* Test accuracy: 0.7519

### 實験2 - 加入learning rate decay call back和調整Adam的learning rate

* Test loss: 0.5792499286174774
* Test accuracy: 0.871